In [1]:
import numpy as np
from skimage import io
import pandas as pd
import os, shutil

In [2]:
steps, R, K, freq = 10000, 12, 0.6, 40
D = 2*R+1
freq=np.pi/freq
init_name='input'
phase = np.asarray(io.imread(f'{init_name}.png'), dtype=np.float32)

In [3]:
phase[np.isnan(phase)]=0
phase=phase-phase.min()
phase=phase/(phase.max()/(2*np.pi))
phase[phase>2*np.pi]=2*np.pi

In [4]:
io.imshow(phase)

/anaconda3/envs/default/lib/python3.7/site-packages/skimage/io/_plugins/matplotlib_plugin.py:78: UserWarning: Float image out of standard range; displaying image with stretched contrast.
  warn("Float image out of standard range; displaying "


In [ ]:
basename='outputs/'
base_this_sim=f"{basename}{init_name}_{K}_{R}"
paths_save = [f"{base_this_sim}/", f"{base_this_sim}_el/"]
folders = [0,1]
for i in folders:
    if os.path.exists(paths_save[i]):
        shutil.rmtree(paths_save[i])
        os.makedirs(paths_save[i])
    else:
        os.makedirs(paths_save[i])

neighb_count = 1/(D*D)
xx, yy = np.meshgrid(np.arange(D)-R,np.arange(D)-R)
xx=xx.flatten()
yy=yy.flatten()
xx=np.delete(xx, D*D//2)
yy=np.delete(yy, D*D//2)
for z in range(steps):
    phase_dif=np.zeros_like(phase)
    elastic_map=np.zeros_like(phase)
    for i in range(D*D-1):
        cur_phase_dif = np.roll(np.roll(np.pad(phase, R, mode='edge'), xx[i], axis=1), yy[i], axis=0)[R:-R,R:-R]-phase
        phase_dif+=np.sin(cur_phase_dif)
        elastic_map+=1 - np.cos(cur_phase_dif)
    phase += K*neighb_count*phase_dif
    phase = phase%(2*np.pi)
    
    io.imsave(f'{paths_save[0]+str(z).zfill(4)}.tif',phase)
    io.imsave(f'{paths_save[1]+str(z).zfill(4)}.tif',elastic_map)